In [1]:
import numpy as np
import subprocess

import tensorflow as tf
import trieste
import gpflow
import timeit

2021-11-15 11:01:35.821773: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-15 11:01:35.821808: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-11-15 11:01:38.188516: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-15 11:01:38.188886: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-15 11:01:38.188939: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.s

In [2]:
def try_get_number(s):
    try:
        float(s)
    except ValueError:
        return None


def single_run(x):
    # we expect `x` to be numpy array (1,D)
    # if it's a tensor, or 2d, convert
    if tf.is_tensor(x):
        x = x.numpy()

    if len(x.shape) == 2:
        x = x[0]

    # first line is the number of elements
    # then one element per line
    np.savetxt("design_vector_0.dat", np.insert(x, 0, len(x), axis=0))

    # invoke Igloo
    subprocess.run(["./Pb_test/run.sh"])

    # both files looks like this:
    # 1
    # 0
    # 0.123456789
    #
    # Meaning of first two lines isn't clear, last line is the value we need
    # Last line can sometimes be missing, or NaN, which means what exactly?
    
    with open('simulation_result_0.dat') as output_file:
        all_lines = output_file.readlines()
        if len(all_lines) != 3:
            objective_value = None
        else:
            objective_value = try_get_number(all_lines[-1])

    with open('simulation_result_1.dat') as output_file:
        all_lines = output_file.readlines()
        if len(all_lines) != 3:
            constraint_value = None
        else:
            constraint_value = try_get_number(all_lines[-1])

    return objective_value, constraint_value

from trieste.data import Dataset

OBJECTIVE = "OBJECTIVE"
CONSTRAINT = "CONSTRAINT"
FAILURE = "FAILURE"

def observer(query_points):
    objective_values = []
    constraint_values = []
    failure_values = []
    for point in query_points:
        ov, cv = single_run(point)
        if ov is None or cv is None:
            # can one be None but not the other? what to do if that happens?
            failure_values.append(0.0)
        else:
            failure_values.append(1.0)
            objective_values.append(ov)
            constraint_values.append(cv)

    return {
        OBJECTIVE: Dataset(query_points, tf.convert_to_tensor(objective_values, dtype=tf.float64)),
        CONSTRAINT: Dataset(query_points, tf.convert_to_tensor(constraint_values, dtype=tf.float64)),
        FAILURE: Dataset(query_points, tf.convert_to_tensor(failure_values, dtype=tf.float64))
    }

In [3]:
search_space = trieste.space.Box(
    [0.02, 0.03, 0.03, 0.03, 0.03, 0.02, 0.01, 0.005, -0.05, -0.06, -0.07, -0.08, -0.08, -0.07, -0.05, -0.03],
    [0.05, 0.08, 0.10, 0.10, 0.10, 0.08, 0.07,  0.05, -0.02, -0.02, -0.03, -0.03, -0.03, -0.01,  0.00,  0.00]
)

In [4]:
num_initial_points = search_space.dimension  # R code uses twice that
initial_query_points = search_space.sample(num_initial_points)

start = timeit.default_timer()
initial_data = observer(initial_query_points)
stop = timeit.default_timer()

print(f"Time for initial points: {stop - start:.0f}s")

MPI initialized
Number of processes: 1
read element : 0
fill element : 0
completed element : 0
read element : 1
fill element : 1
completed element : 1
read element : 2
fill element : 2
completed element : 2
read element : 3
fill element : 3
completed element : 3
read element : 4
fill element : 4
completed element : 4
read element : 5
fill element : 5
completed element : 5
read element : 6
fill element : 6
completed element : 6
read element : 7
fill element : 7
completed element : 7
read element : 8
fill element : 8
completed element : 8
read element : 9
fill element : 9
completed element : 9
read element : 10
fill element : 10
completed element : 10
read element : 11
fill element : 11
completed element : 11
read element : 12
fill element : 12
completed element : 12
read element : 13
fill element : 13
completed element : 13
read element : 14
fill element : 14
completed element : 14
read element : 15
fill element : 15
completed element : 15
read element : 16
fill element : 16
completed e

MPI finalized
MPI initialized
Number of processes: 1
construction from mesh file: mesh_refined.dat
elements list : ok
dof mapping : ok
faces list : ok
boundaries list : ok

number of partitions: 1
number of elements: 272
number of interior faces: 511
number of boundary faces: 66
number of variables: 4
element degree: 3
number of control points per element: 16
number of dofs: 4352
number of Gauss points per element: 16

wall refinement: 1 number of refined elements: 16

Removing haning nodes ...
... Done
>>> mesh refinement process: selection
Mean error: 0.125
blocking configurations: 0
>>> selection done

>>> mesh refinement process: refinement
Number of elements - previous: 272, new: 340, active: 306
Number of faces - previous: 511, new: 611, active: 578
Number of boundary faces - previous: 66, new: 70, active: 68
State vector size  - previous: 17408, new: 21760
Number of active DOFs: 4896
>>> refinement done

wall refinement: 2 number of refined elements: 16

Removing haning nodes ..

MPI initialized
Number of processes: 1
construction from mesh file: mesh.dat
elements list : ok
dof mapping : ok
faces list : ok
boundaries list : ok

number of partitions: 1
number of elements: 36
number of interior faces: 54
number of boundary faces: 36
number of variables: 4
element degree: 3
number of control points per element: 16
number of dofs: 576
number of Gauss points per element: 16


Box refinement:
1 3
-20 20


Refinement direction: 0

>>> mesh refinement process: selection
Mean error: 0.111111
blocking configurations: 0
>>> selection done

>>> mesh refinement process: refinement
Number of elements - previous: 36, new: 44, active: 40
Number of faces - previous: 54, new: 64, active: 61
Number of boundary faces - previous: 36, new: 40, active: 38
State vector size  - previous: 2304, new: 2816
Number of active DOFs: 640
>>> refinement done


Box refinement:
1 2
-20 20


Refinement direction: 0

>>> mesh refinement process: selection
Mean error: 0.1
blocking configurations: 0


MPI initialized
Number of processes: 1
construction from mesh file: mesh_refined.dat
elements list : ok
dof mapping : ok
faces list : ok
boundaries list : ok

number of partitions: 1
number of elements: 340
number of interior faces: 645
number of boundary faces: 70
number of variables: 4
element degree: 3
number of control points per element: 16
number of dofs: 5440
number of Gauss points per element: 16

MPI finalized
MPI initialized
Number of processes: 1
construction from mesh file: mesh_elevated.dat
elements list : ok
dof mapping : ok
faces list : ok
boundaries list : ok

number of partitions: 1
number of elements: 340
number of interior faces: 645
number of boundary faces: 70
number of variables: 4
element degree: 3
number of control points per element: 16
number of dofs: 5440
number of Gauss points per element: 16

MPI finalized
MPI initialized
Number of processes: 1
construction from mesh file: mesh_deformed.dat
elements list : ok
dof mapping : ok
faces list : ok
boundaries list

ValueError: could not convert string to float: 

In [ ]:
# this needs much more work, models for constraints and failures
# but let's get past the initial data phase first

from trieste.models.gpflow import GPflowModelConfig


def create_bo_model(data):
    variance = tf.math.reduce_variance(initial_data[OBJECTIVE].observations)
    lengthscale = 0.01 * np.ones(search_space.dimension, dtype=gpflow.default_float())
    kernel = gpflow.kernels.Matern52(variance=variance, lengthscales=lengthscale)
    jitter = gpflow.kernels.White(1e-12)
    gpr = gpflow.models.GPR(data.astuple(), kernel + jitter, noise_variance=1e-5)
    gpflow.set_trainable(gpr.likelihood, False)
    return trieste.models.create_model(GPflowModelConfig(**{
        "model": gpr,
        "optimizer": gpflow.optimizers.Scipy(),
        "optimizer_args": {
            "minimize_args": {"options": dict(maxiter=100)},
        },
    }))

initial_models = trieste.utils.map_values(create_bo_model, initial_data)

In [ ]:
from trieste.acquisition.rule import EfficientGlobalOptimization

pof = trieste.acquisition.ProbabilityOfFeasibility(threshold=0.0)  # g(x) <= 0, according to https://rdrr.io/cran/DiceOptim/man/EGO.cst.html
eci = trieste.acquisition.ExpectedConstrainedImprovement(OBJECTIVE, pof.using(CONSTRAINT))
rule = EfficientGlobalOptimization(eci)

In [ ]:
num_steps = 20
bo = trieste.bayesian_optimizer.BayesianOptimizer(observer, search_space)

start = timeit.default_timer()
data = bo.optimize(num_steps, initial_data, initial_models, rule, track_state=False).try_get_final_datasets()
stop = timeit.default_timer()

print(f"Time for initial points: {stop - start:.0f}s")